# PyGrid: Remote Inference - Data Owner

<img src="../../../docs/img/pygrid_logo.png" align="center"/>

The ability to evaluate custom models, using private datasets without having access to them; is a powerful idea that will change the way we interact with data during a machine learning workflow. PySyft and PyGrid offer the ability to run inferences remotely by using a variety of technologies and applications.

In this notebook series, we'll be covering all the nuances of this process, showing how to send private datasets _(as Data Owner)_, and how to perform remote computation using private environments _(as Data Scientist)_.

The main goal of these notebooks is to explore different techniques and technologies that can finally make **private data _"accessible"_ to Data Scientists, while also providing Data Owners with total control of their data.**

**NOTE**: _This notebook was designed to be executed in pair with the [PyGrid Remote Inference - Data Scientist](./PyGrid%20Remote%20Inference%20-%20Data%20Scientist.ipynb) notebook. In order to reproduce it properly, follow the checkpoints and instructions described in the next sections._

### Overview

- [**Initial Setup**](#initial-setup)
- [**Hosting Datasets**](#hosting-datasets)
- [**Create a Compliance Officer Account**](#create-a-compliance-officer-account)
- [**Data Access Triage**](#data-access-request-triage)

<a id="initial-setup"></a>
## Initial Setup
The first thing to do after deploying a domain, is to set its initial configuration. Here we can define some values and flags that will change how the domain will behave. We also use this step to create the owner account, which will have full control and access to the platform. As long as this step is not executed, all other endpoints/features will be blocked. **NOTE**: Setup is also possible via a web interface.

### Importing libs

In [ ]:
from syft.grid.client.client import connect  # Method used to connect with the domain.
from syft.grid.client.grid_connection import (
    GridHTTPConnection,
)  # Protocol used to talk with the domain

import syft as sy
import torch as th

# Set logging level
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
PYGRID_PORT = 5000

In [ ]:
# 1 - Since we still don't have any account registered,
# we can connect with the domain without credentials.
domain_client = connect(
    url=f"http://localhost:{PYGRID_PORT}",  # Domain Address
    conn_type=GridHTTPConnection,
)  # HTTP Connection Protocol

# 2 - Set the domain initial configuration
# - Owner's email/password
# - Domain Name used to identify this owner.
# - Random token generated during deployment
domain_client.setup(
    email="owner@myorg.com",
    password="owerpwd",
    domain_name="OpenMined Domain",
    token="9G9MJ06OQH",
)

In [ ]:
# 3 - Now, we have an account registered!
# Let's connect with the domain as using owner credentials.
domain = connect(
    url=f"http://localhost:{PYGRID_PORT}",  # Domain Address
    credentials={"email": "owner@myorg.com", "password": "owerpwd"},
    conn_type=GridHTTPConnection,
)  # HTTP Connection Protocol

_PS : For didatic purposes, we have reduced the number of parameters used in the "setup" process. But the domain also accepts a set of optional parameters. If you want to know more about the initial setup process, it's highly recommended to read our roadmap._

<a id="hosting-datasets"></a>
## Hosting Datasets

As a domain owner, we're also able to upload new datasets to the domain. We can do it by using two different approaches:

- **Tensor Data Structures**: Using the torch tensor structures and the default PySyft API
- **Structured and compressed CSV folder** : We can upload a _.tar.gz_ file  by using the PyGrid API.

We'll be covering the second one, sending a compressed file containing a structured folder with different CSV files + metadata information about the dataset _(manifest, description and tags)_.

In [ ]:
dataset = domain.datasets.create("diabetes-dataset.tar.gz")
dataset.pandas

Now, the dataset is available to be used remotely by a data scientist.

<a id="create-a-compliance-officer-account"></a>
## Create a Compliance Officer Account
By default, PyGrid initializes with 4 pre-set roles that can be attached to the users. These roles are used to define what a user is allowed or not allowed to do when logged-in. You can check its permissions by executing this code snippet.

### Roles

In [ ]:
domain.roles

In [ ]:
domain.roles.all(pandas=True)

These roles are not static _(excluding owner role)_ and can be updated or deleted if necessary. You can also create new ones if needed.

Using owner permissions, you can also check and change the roles used by each user registered in the domain.

In [ ]:
domain.users.all(pandas=True)

### User Creation

PyGrid allow people to create new accounts by themselves, therefore, these accounts will always be  attached to the User role _(without any management permissions)_. Considering the default roles, only the Owner and Administrator accounts would be able to create new accounts attached with different permissions.<br><br>
For our use case, we'll create a **compliance officer** account. This role is commonly assigned for users who will **evaluate data access requests**, approving or rejecting them. Since we're logged-in as a owner, we have all the requirements.

In [ ]:
domain.users.create(
    email="compliance@myorg.com", password="pwd123", role="Compliance Officer"
)

In [ ]:
domain.users.all(pandas=True)

### Login as a Compliance Officer
To be accurate to this use case, we'll log-in as compliance officer.<br><br>
**From now on, we'll use only compliance officer credentials to go through this use case.**

In [ ]:
cp_officer = connect(
    url=f"http://localhost:{PYGRID_PORT}",  # Domain Address
    credentials={"email": "compliance@myorg.com", "password": "pwd123"},
    conn_type=GridHTTPConnection,
)  # HTTP Connection Protocol

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint : Now STOP and run the Data Scientist notebook until the next checkpoint.

<a id="data-access-request-triage"></a>
## Data Access Triage

Empowered by the data access management, the compliance officer the evaluate each request accepting or denying them.

In [ ]:
cp_officer.requests.pandas

In [ ]:
cp_officer.requests[0].accept()

## Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Take our FREE Video Course "The Private AI Series"
Learn how privacy technology is changing our world and how you can lead the charge.
We cover non technical concepts about structured transparency, as well as deep dive into the technical aspects of various Cryptographic technologies and how to use them with Syft and Grid.
* [📺 Video Course](https://courses.openmined.org/)

### Star PySyft and PyGrid on GitHub
The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

* [⭐️ Star PySyft](https://github.com/OpenMined/PySyft)
* [⭐️ Star PyGrid](https://github.com/OpenMined/PyGrid)

### Join our Slack!
The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at http://slack.openmined.org

### Join a Code Project!
The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

* [PySyft Good First Issue Tickets](https://github.com/OpenMined/PySyft/labels/Good%20first%20issue%20%3Amortar_board%3A)
* [PyGrid Good First Issue Tickets](https://github.com/OpenMined/PyGrid/labels/good%20first%20issue)

### Donate
If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

* [OpenMined's Open Collective Page](https://opencollective.com/openmined)